#### Libraries

In [148]:
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import sklearn
from ast import literal_eval

In [149]:
data = pd.read_csv('./02_Data_10-25/2A_data-output.csv', index_col=0, converters={'production_companies': literal_eval}, parse_dates=['release_date'])
data = data.drop([267])

In [150]:
data.head(6)

,id,release,release_date,opening_revenue,percent_of_total_revenue,opening_theaters,genre_ids,original_language,popularity,vote_average,...,writer,writer_xp_median,writer_xp_sum,writer_rating_median,writer_rating_max,production_room,writers_room,sound_room,crew,based_on_novel
0,59,A History of Violence,2005-09-23,"$364,000",1.20%,14,"[18, 53, 80]",en,23.726,7.2,...,306.0,3.5,7.0,4.7000,5.800,13.0,3.0,18.0,73.0,0.0
1,8976,A Lot Like Love,2005-04-21,"$7,576,593",34.70%,"2,502","[35, 18, 10749]",en,12.853,6.8,...,56459.0,NaN,0.0,NaN,NaN,5.0,1.0,13.0,36.0,0.0
2,10077,A Sound of Thunder,2005-05-15,"$917,398",48.30%,816,"[53, 878, 12, 28]",en,17.001,4.8,...,53297.0,6.0,62.0,5.9565,10.000,31.0,4.0,16.0,133.0,0.0
3,8202,Æon Flux,2005-11-30,"$12,661,112",48.90%,"2,608","[878, 28, 53]",en,22.860,5.6,...,54047.0,4.0,17.0,6.6000,10.000,33.0,3.0,21.0,187.0,0.0
4,22559,Aliens of the Deep,2005-01-28,"$479,368",5.30%,27,[99],en,12.978,6.1,...,1192052.0,7.0,7.0,6.9000,7.900,5.0,1.0,1.0,13.0,0.0
5,12142,Alone in the Dark,2005-01-28,"$2,834,421",54.70%,"2,124","[28, 14, 27]",en,14.815,3.2,...,40607.0,1.0,2.0,4.5355,5.971,24.0,3.0,15.0,143.0,0.0


#### Drop Features & Observations Outside Scope of Study

In [136]:
data.columns

Index(['id', 'release', 'release_date', 'opening_revenue',
       'percent_of_total_revenue', 'opening_theaters', 'genre_ids',
       'original_language', 'popularity', 'vote_average', 'vote_count',
       'budget', 'genres', 'imdb_id', 'origin_country', 'production_companies',
       'production_countries', 'revenue', 'runtime',
       'belongs_to_collection.id', 'belongs_to_collection.name',
       'position_in_series', 'previous_film_release', 'previous_film_rating',
       'cast_xp_median', 'cast_xp_sum', 'cast_rating_max', 'director',
       'director_xp', 'director_rating_median', 'director_rating_max',
       'writer', 'writer_xp_median', 'writer_xp_sum', 'writer_rating_median',
       'writer_rating_max', 'production_room', 'writers_room', 'sound_room',
       'crew', 'based_on_novel'],
      dtype='object')

In [137]:
baseline = data.drop(columns = ['percent_of_total_revenue', #posterior information to opening weekend focus
                                'opening_theaters', #alternative target to opening_revenue, if necessary
                                'popularity', #manufactured by TMDB, includes posterior information to opening weekend focus
                                'vote_average', #alternative target to opening_revenue; however, includes posterior information to opening weekend focus
                                'vote_count', #alternative target to opening_revenue; however, includes posterior information to opening weekend focus
                                'imdb_id', #crosswalk to imdb dataset, which we didn't use
                                'revenue', #alternative target
                                'writer',
                                'director',
                                'original_language'
                              ]) 


In [138]:
baseline = baseline[baseline.runtime > 60]

In [139]:
baseline.head(5)

,id,release,release_date,opening_revenue,genre_ids,budget,genres,origin_country,production_companies,production_countries,...,director_rating_max,writer_xp_median,writer_xp_sum,writer_rating_median,writer_rating_max,production_room,writers_room,sound_room,crew,based_on_novel
0,59,A History of Violence,2005-09-23,"$364,000","[18, 53, 80]",32000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",['US'],"[{'id': 12, 'logo_path': '/2ycs64eqV5rqKYHyQK0...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",...,7.411,3.5,7.0,4.7000,5.8,13.0,3.0,18.0,73.0,0.0
1,8976,A Lot Like Love,2005-04-21,"$7,576,593","[35, 18, 10749]",30000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",['US'],"[{'id': 2977, 'logo_path': None, 'name': 'Kevi...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,6.532,NaN,0.0,NaN,NaN,5.0,1.0,13.0,36.0,0.0
2,10077,A Sound of Thunder,2005-05-15,"$917,398","[53, 878, 12, 28]",80000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...","['GB', 'US']","[{'id': 1403, 'logo_path': '/pMTQNUhHCwr39CsS7...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'...",...,7.500,6.0,62.0,5.9565,10.0,31.0,4.0,16.0,133.0,0.0
3,8202,Æon Flux,2005-11-30,"$12,661,112","[878, 28, 53]",62000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",['US'],"[{'id': 4, 'logo_path': '/gz66EfNoYPqHTYI4q9UE...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,6.300,4.0,17.0,6.6000,10.0,33.0,3.0,21.0,187.0,0.0
4,22559,Aliens of the Deep,2005-01-28,"$479,368",[99],0,"[{'id': 99, 'name': 'Documentary'}]",['US'],"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,8.100,7.0,7.0,6.9000,7.9,5.0,1.0,1.0,13.0,0.0


#### Correcting Data Type

In [140]:
baseline['opening_revenue'] = baseline['opening_revenue'].replace('[\$,]', '', regex=True).replace('-', 0, regex=False).astype(float)
baseline['release_date'] = pd.to_datetime(baseline['release_date'], format='%Y-%m-%d')

#### Handling Missing Values

*Target Variable*

In [141]:
baseline[baseline.opening_revenue == 0]

,id,release,release_date,opening_revenue,genre_ids,budget,genres,origin_country,production_companies,production_countries,...,director_rating_max,writer_xp_median,writer_xp_sum,writer_rating_median,writer_rating_max,production_room,writers_room,sound_room,crew,based_on_novel
51,10982,Hoodwinked!,2005-12-16,0.0,"[16, 35, 10751, 80]",8000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",['US'],"[{'id': 7580, 'logo_path': None, 'name': 'Kanb...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,0.000,5.0,10.0,0.0,0.000,4.0,3.0,7.0,18.0,0.0
147,9912,The World's Fastest Indian,2005-10-12,0.0,"[18, 12, 36]",25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...","['NZ', 'CH', 'US']","[{'id': 294, 'logo_path': '/a05DY2oSeUOVFDM61R...","[{'iso_3166_1': 'NZ', 'name': 'New Zealand'}, ...",...,7.027,58.0,58.0,5.8,7.027,14.0,2.0,12.0,52.0,0.0


Confirmed with Box Office Mojo source that these films were recorded with null revenue statistics. Provided the limitation of the problem, and the amount of data, we'll choose to remove these nulls rather than try and estimate / imput

In [142]:
baseline = baseline[baseline.opening_revenue > 0]

*Features*

In [143]:
pd.DataFrame(baseline.isna().sum()).sort_values(by = 0, ascending = False).head(15)

,0
belongs_to_collection.name,394
belongs_to_collection.id,394
writer_xp_median,58
writer_rating_max,58
writer_rating_median,58
director_rating_median,24
director_rating_max,24
cast_xp_median,5
cast_rating_max,5
writer_xp_sum,4


belongs_to_collection features controlling for is_series - we'll eventually eliminate this column, once processed in feature engineering. 

Other features are indicative of crew with no prior experience to the film - filling their values with 0. We'll follow up on this in feature engineering as well, to potentially build a binary indicator feature

In [144]:
baseline.writer_rating_max = baseline.writer_rating_max.fillna(0)
baseline.writer_rating_median = baseline.writer_rating_median.fillna(0)
baseline.writer_xp_median = baseline.writer_xp_median.fillna(0)
baseline.director_rating_max = baseline.director_rating_max.fillna(0)
baseline.director_rating_median = baseline.director_rating_median.fillna(0)
baseline.cast_xp_median = baseline.cast_xp_median.fillna(0)
baseline.cast_rating_max = baseline.cast_rating_max.fillna(0)
baseline.writer_xp_sum = baseline.writer_xp_sum.fillna(0)

In [147]:
baseline.to_csv('./02_Data_10-25/2B_data-output.csv')